[View in Colaboratory](https://colab.research.google.com/github/mohammed-Emad/add_faces_to_svm_class/blob/master/add_clasifir_class.ipynb)

#There was a problem with the environment variable for this. The easiest solution was to put all files in one path
**Thus an error was exceeded**

**ImportError: No module named 'facenet'**

 **and**
 


```
import align.detect_face 
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
ImportError: No module named 'align'
```


Now upload these files from my device

In [2]:
from google.colab import files
uploded=files.upload()

Saving src_facenet.zip to src_facenet.zip


In [3]:
%mkdir facenet
!unzip src_facenet.zip -d facenet
!rm src_facenet.zip
!ls 

Archive:  src_facenet.zip
 extracting: facenet/__init__.py     
  inflating: facenet/align_dataset_mtcnn.py  
  inflating: facenet/calculate_filtering_metrics.py  
  inflating: facenet/classifier.py   
  inflating: facenet/compare.py      
  inflating: facenet/decode_msceleb_dataset.py  
  inflating: facenet/det1.npy        
  inflating: facenet/det2.npy        
  inflating: facenet/det3.npy        
  inflating: facenet/detect_face.py  
  inflating: facenet/download_and_extract.py  
  inflating: facenet/facenet.py      
  inflating: facenet/freeze_graph.py  
  inflating: facenet/lfw.py          
  inflating: facenet/mmc.py          
  inflating: facenet/train_softmax.py  
  inflating: facenet/train_tripletloss.py  
  inflating: facenet/validate_on_lfw.py  
datalab  facenet


#download data lfw and Extraction

In [0]:
%cd ~/
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
  
!mkdir  -p ~/datasets/lfw/raw
!tar xvf ~/lfw.tgz -C ~/datasets/lfw/raw --strip-components=1

#download model

In [0]:
     !pip install -U -q PyDrive ## you will have install for every colab session

     from pydrive.auth import GoogleAuth
     from pydrive.drive import GoogleDrive
     from google.colab import auth
     from oauth2client.client import GoogleCredentials

     # 1. Authenticate and create the PyDrive client.
     auth.authenticate_user()
     gauth = GoogleAuth()
     gauth.credentials = GoogleCredentials.get_application_default()
     drive = GoogleDrive(gauth)

In [8]:
import json
json_import = drive.CreateFile({'id':'1EXPBSXwTaqrSC0OhUdXNmKSh9qJUQ55-'})
json_import.GetContentFile('20180402-114759.zip')
!ls

20180402-114759.zip  datalab  datasets	facenet  lfw.tgz


In [9]:
!unzip 20180402-114759.zip -d datasets

Archive:  20180402-114759.zip
   creating: datasets/20180402-114759/
  inflating: datasets/20180402-114759/model-20180402-114759.meta  
  inflating: datasets/20180402-114759/20180402-114759.pb  
  inflating: datasets/20180402-114759/model-20180402-114759.ckpt-275.data-00000-of-00001  
  inflating: datasets/20180402-114759/model-20180402-114759.ckpt-275.index  


In [0]:
!rm *.zip 
!rm *.tgz

In [11]:
!mkdir my_model
!ls

datalab  datasets  facenet  my_model


In [12]:
%cd facenet
!ls

/content/facenet
align_dataset_mtcnn.py		det2.npy		 __init__.py
calculate_filtering_metrics.py	det3.npy		 lfw.py
classifier.py			detect_face.py		 mmc.py
compare.py			download_and_extract.py  train_softmax.py
decode_msceleb_dataset.py	facenet.py		 train_tripletloss.py
det1.npy			freeze_graph.py		 validate_on_lfw.py


#align dataset
*see [Validate-on-lfw](https://github.com/davidsandberg/facenet/wiki/Validate-on-lfw)*

In [0]:
!python align_dataset_mtcnn.py \
~/datasets/lfw/raw \
~/datasets/lfw/lfw_mtcnnpy_160 \
--image_size 160 \
--margin 32 \
--random_order \
--gpu_memory_fraction 0.25 \


In [81]:
!wget https://raw.githubusercontent.com/mohammed-Emad/add_faces_to_svm_class/master/my_clasifiy.py

--2018-06-16 09:34:16--  https://raw.githubusercontent.com/mohammed-Emad/add_faces_to_svm_class/master/my_clasifiy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11558 (11K) [text/plain]
Saving to: ‘my_clasifiy.py’

my_clasifiy.py      100%[===================>]  11.29K  --.-KB/s    in 0s      

2018-06-16 09:34:16 (31.1 MB/s) - ‘my_clasifiy.py’ saved [11558/11558]



#train normal

In [37]:
!python my_clasifiy.py \
TRAIN  \
~/datasets/lfw/lfw_mtcnnpy_160 \
~/datasets/20180402-114759/20180402-114759.pb  \
~/my_model/lfw_classifier.pkl \
--batch_size 1000 \
--min_nrof_images_per_class 40 \
--nrof_train_images_per_class 35 \
--use_split_dataset


my_clasifiy.py:62: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-06-16 05:21:03.224245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-06-16 05:21:03.224786: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-16 05:21:03.224827: I tensorflow/c

#train  incremental

In [19]:
!python my_clasifiy.py \
TRAIN  \
--training_type incremental \
~/datasets/lfw/lfw_mtcnnpy_160 \
~/datasets/20180402-114759/20180402-114759.pb  \
~/my_model/lfw_classifier.pkl \
--features_filename ~/my_model/all_features.pkl \
--batch_size 1000 \
--min_nrof_images_per_class 40 \
--nrof_train_images_per_class 35 \
--use_split_dataset




my_clasifiy.py:62: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-06-16 04:52:42.242867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-06-16 04:52:42.243541: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-16 04:52:42.243613: I tensorflow/c

#clasifiy a data lfw

In [100]:
!python my_clasifiy.py \
CLASSIFY \
~/datasets/lfw/lfw_mtcnnpy_160 \
~/datasets/20180402-114759/20180402-114759.pb  \
~/my_model/lfw_classifier.pkl \
--batch_size 1000 \
--min_nrof_images_per_class 40 \
--nrof_train_images_per_class 35 \
--use_split_dataset

my_clasifiy.py:62: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-06-16 11:17:42.943983: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-06-16 11:17:42.944616: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-16 11:17:42.944665: I tensorflow/c

#Overview of file features

In [21]:
!ls ~/my_model/

all_features.pkl  lfw_classifier.pkl


In [0]:
import os ,pickle
# model all features
with open('/content/my_model/all_features.pkl', 'rb') as infile:
      (emb_array_old ,labels_old ,class_names_old) = pickle.load(infile)
          

In [0]:
# model svc & svm
with open('/content/my_model/lfw_classifier.pkl', 'rb') as infile:
     (model, class_names) = pickle.load(infile)


In [63]:
print(model)
print(class_names)

print("class_names",len(class_names))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
['Ariel Sharon', 'Arnold Schwarzenegger', 'Colin Powell', 'Donald Rumsfeld', 'George W Bush', 'Gerhard Schroeder', 'Gloria Macapagal Arroyo', 'Hugo Chavez', 'Jacques Chirac', 'Jean Chretien', 'Jennifer Capriati', 'John Ashcroft', 'Junichiro Koizumi', 'Laura Bush', 'Lleyton Hewitt', 'Luiz Inacio Lula da Silva', 'Serena Williams', 'Tony Blair', 'Vladimir Putin']
class_names 19


In [40]:
print(len(class_names_old))
print(len(labels_old))
print(len(emb_array_old))
print("\n")

print(class_names_old)
print(labels_old)
print(emb_array_old)

19
665
665


['Ariel Sharon', 'Arnold Schwarzenegger', 'Colin Powell', 'Donald Rumsfeld', 'George W Bush', 'Gerhard Schroeder', 'Gloria Macapagal Arroyo', 'Hugo Chavez', 'Jacques Chirac', 'Jean Chretien', 'Jennifer Capriati', 'John Ashcroft', 'Junichiro Koizumi', 'Laura Bush', 'Lleyton Hewitt', 'Luiz Inacio Lula da Silva', 'Serena Williams', 'Tony Blair', 'Vladimir Putin']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

#See the top here to find the number of fonts in the file attributes from the names to the features of the images and the order


```
print(class_names_old)
print(labels_old)
print(emb_array_old)

```


```
19
665
665

```




#my edit to clasifir file
#  
# my lines as 



```
#-----<--new
```



In [0]:
import tensorflow as tf
import numpy as np
import argparse
import facenet
import os
import sys
import math
import pickle
from sklearn.svm import SVC

In [54]:

def main(args):
  
    with tf.Graph().as_default():
      
        with tf.Session() as sess:
            
            np.random.seed(seed=args.seed)

            if args.use_split_dataset:
                dataset_tmp = facenet.get_dataset(args.data_dir)

                train_set, test_set = split_dataset(dataset_tmp, args.min_nrof_images_per_class, args.nrof_train_images_per_class)
                if (args.mode=='TRAIN'):
                    dataset = train_set
                elif (args.mode=='CLASSIFY'):
                    dataset = test_set
            else:
                dataset = facenet.get_dataset(args.data_dir)

            # Check that there are at least one training image per class
            for cls in dataset:
                assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset') 

            stack_old_data = [] #----<--new
            if os.path.isfile(args.features_filename) and (args.training_type == 'incremental'): # -----<--new
               
               stack_old_data ,dataset = Remove_duplicate_names(args ,dataset)                   #----<--new

            paths, labels = facenet.get_image_paths_and_labels(dataset)
            
            print('Number of classes: %d' % len(dataset))
            print('Number of images: %d' % len(paths))
            
            # Load the model
            print('Loading feature extraction model')
            facenet.load_model(args.model)
            
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            embedding_size = embeddings.get_shape()[1]
            
            # Run forward pass to calculate embeddings
            print('Calculating features for images')
            nrof_images = len(paths)
            nrof_batches_per_epoch = int(math.ceil(1.0*nrof_images / args.batch_size))
            emb_array = np.zeros((nrof_images, embedding_size))
            for i in range(nrof_batches_per_epoch):
                start_index = i*args.batch_size
                end_index = min((i+1)*args.batch_size, nrof_images)
                paths_batch = paths[start_index:end_index]
                images = facenet.load_data(paths_batch, False, False, args.image_size)
                feed_dict = { images_placeholder:images, phase_train_placeholder:False }
                emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)
            
            classifier_filename_exp = os.path.expanduser(args.classifier_filename)

            if (args.mode=='TRAIN'):
                # Train classifier
                print('Training classifier')
                # Create a list of class names
                class_names = [ cls.name.replace('_', ' ') for cls in dataset]

                if (args.training_type == 'incremental'): #-----<--new
                   emb_array ,labels ,class_names = Incremental_training(args ,stack_old_data, emb_array ,labels ,class_names) #----<--new

                model = SVC(kernel='linear', probability=True)
                model.fit(emb_array, labels)
                
                # Saving classifier model
                with open(classifier_filename_exp, 'wb') as outfile:
                    pickle.dump((model, class_names), outfile)
                print('Saved classifier model to file "%s"' % classifier_filename_exp)
                
            elif (args.mode=='CLASSIFY'):
                # Classify images
                print('Testing classifier')
                with open(classifier_filename_exp, 'rb') as infile:
                    (model, class_names) = pickle.load(infile)

                print('Loaded classifier model from file "%s"' % classifier_filename_exp)

                predictions = model.predict_proba(emb_array)
                best_class_indices = np.argmax(predictions, axis=1)
                best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
                
                for i in range(len(best_class_indices)):
                    print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))
                    
                accuracy = np.mean(np.equal(best_class_indices, labels))
                print('Accuracy: %.3f' % accuracy)

<ipython-input-54-4a3c07cb82c9>:23: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')


#My functions

**When you start adding new people to our people base
A big problem may occur when we add people already in the previous data.
Or the inclusion of persons with repeated names**

**Here we can use a function to remove people who have been found in previous data from the current image base.
Where the "*Remove_duplicate_names()*" function gets the new DataSet
The same data is returned after deleting the names**

**The comparison here is done by uploading the feature model and then searching inside the saved names
On this basis, people are bypassed**

**There is something else to be brought back
We are here when we wanted to discover duplicate names we uploaded the features file
And we already have the full elements
Here we are restoring those items so that they have to be loaded again
Where we will need those elements later to add new data to them **

In [0]:
def Remove_duplicate_names(args ,dataset):
    image_del = 0
    name_deleted = []
    stak_host = []
    dataset_n =[]
    features_filename = os.path.expanduser(args.features_filename)

    print("load_features from file :'%s'" % args.features_filename)
    with open(features_filename, 'rb') as infile:
             (emb_old ,labels_old ,class_old) = pickle.load(infile)

    if len(class_old) > 0:
      for cls in dataset:
          name = cls.name.replace('_', ' ')    #get the class name
          if name in class_old:                #find
             name_deleted.append(name) 
             image_del +=len(cls.image_paths)

          else:
             dataset_n.append(facenet.ImageClass(cls.name, cls.image_paths))

    if image_del > 0:

       #Remove duplicate names in the deletion list
       ResultList = sorted(set(name_deleted), key=lambda x:name_deleted.index(x))
       print("About (%s) files of data were deleted for about '%s' people whose names already existd"% image_del ,len(ResultList))

    else:
       print("No one has been deleted(%s)" % len(name_deleted))
    stak_host = emb_old ,labels_old ,class_old
    return  stak_host ,datasetS


#Incremental_training (fun!)

**The first thing here is to get old data
Then the new data
There is a requirement to make sure that the features file is already in our possession and at least one person !, although this is not logical.
When there are people in the old features file, the features of the new people are added to them with names and proper order
This happens in just four lines.**

**We note that if there are no people in the old data or file does not exist,
and have already activated (--training_type incremental)**

**The current or new data will be saved in a new features file, depending on the name of the file passed in (--features_filename)**

**For the rest of the details, visit the following location
https://github.com/mohammed-Emad/add_faces_to_svm_class**

In [0]:
def Incremental_training(args ,stack_old_data ,emb_n ,labels_n ,class_n): #-----<-- new
    
    features_filename = os.path.expanduser(args.features_filename)

    
    if os.path.isfile(args.features_filename) :

       emb_old ,labels_old ,class_old = stack_old_data
       if len(class_old) >0:
          print("start Incremental_training")
          labels_n = np.array(labels_n) + max(labels_old) +1                      # move numbers new list to old list end
          labels_n = np.concatenate((labels_old ,labels_n.tolist())).tolist()     #add new labels to old labels
          class_n = np.concatenate((class_old ,class_n))                          #append new class name to old class names
          emb_n = np.concatenate((emb_old ,emb_n))                                #append new image feature to old images feature
       
    else:
        print("not found file!:%s and start not Incremental_training" %args.features_filename)

    with open(features_filename, 'wb') as outfile:
         pickle.dump((emb_n ,labels_n ,class_n), outfile, pickle.HIGHEST_PROTOCOL)

    print('Saved a features as model to file "%s"' % args.features_filename)
    return emb_n ,labels_n ,class_n

#done edit

#The following functions are from within the [facenet](https://github.com/davidsandberg/facenet/blob/master/src/classifier.py) project

In [0]:
def split_dataset(dataset, min_nrof_images_per_class, nrof_train_images_per_class):
    train_set = []
    test_set = []
    for cls in dataset:
        paths = cls.image_paths
        # Remove classes with less than min_nrof_images_per_class
        if len(paths)>=min_nrof_images_per_class:
            np.random.shuffle(paths)
            train_set.append(facenet.ImageClass(cls.name, paths[:nrof_train_images_per_class]))
            test_set.append(facenet.ImageClass(cls.name, paths[nrof_train_images_per_class:]))
    return train_set, test_set


#But here we added two orders as argparse
**They are**
**--training_type**

**and**

**--features_filename**

In [0]:

def parse_arguments(argv):
    parser = argparse.ArgumentParser()
    
    parser.add_argument('mode', type=str, choices=['TRAIN', 'CLASSIFY'],
        help='Indicates if a new classifier should be trained or a classification ' + 
        'model should be used for classification', default='CLASSIFY')

    parser.add_argument('--training_type', type=str, choices=['incremental', 'normal'], #------------<--new1
        help='Indicates whether you want to gradually train your data Or normal training.' + 
        'In the extra training you will need to provide the location of the features file.' +
        'informing you that it will be overwritten every time.', default='normal')

    parser.add_argument('data_dir', type=str,
        help='Path to the data directory containing aligned LFW face patches.')
    parser.add_argument('model', type=str, 
        help='Could be either a directory containing the meta_file and ckpt_file or a model protobuf (.pb) file')
    parser.add_argument('classifier_filename', 
        help='Classifier model file name as a pickle (.pkl) file. ' + 
        'For training this is the output and for classification this is an input.')

    parser.add_argument('--features_filename', # --------------<<----new2
        help='Features extracted file name as a pickle (.pkl) file. ' + 
        'path and name to file Features extracted and classes names and lable.', default='none')

    parser.add_argument('--use_split_dataset', 
        help='Indicates that the dataset specified by data_dir should be split into a training and test set. ' +  
        'Otherwise a separate test set can be specified using the test_data_dir option.', action='store_true')
    parser.add_argument('--test_data_dir', type=str,
        help='Path to the test data directory containing aligned images used for testing.')
    parser.add_argument('--batch_size', type=int,
        help='Number of images to process in a batch.', default=90)
    parser.add_argument('--image_size', type=int,
        help='Image size (height, width) in pixels.', default=160)
    parser.add_argument('--seed', type=int,
        help='Random seed.', default=666)
    parser.add_argument('--min_nrof_images_per_class', type=int,
        help='Only include classes with at least this number of images in the dataset', default=20)
    parser.add_argument('--nrof_train_images_per_class', type=int,
        help='Use this number of images from each class for training and the rest for testing', default=10)
    
    return parser.parse_args(argv)

In [0]:
#if __name__ == '__main__':
#    main(parse_arguments(sys.argv[1:]))

#  
#Now start training and test on new data

#First load the data
**These are images of three people who are not in our database**

**At the test, we will test a photo collection of 3 the new people
and Three people were already trained earlier
The names will be as follows**


```
# This is formatted as data
new_dataset/
___________train/
________________person_new_A/
____________________________/*.jpg
________________person_new_B/
____________________________/*.jpg
________________person_new_C/
____________________________/*.jpg
.
___________test/
________________person_new_A/
____________________________/*.jpg
________________person_new_B/
____________________________/*.jpg
________________person_new_C/
____________________________/*.jpg
.
________________person_old_A/
____________________________/*.jpg
________________person_old_B/
____________________________/*.jpg
________________person_old_C/
____________________________/*.jpg

```



In [70]:
#dawnload data
!wget https://github.com/mohammed-Emad/add_faces_to_svm_class/raw/master/new_dataset.zip
!unzip new_dataset.zip
!rm new_dataset.zip

--2018-06-16 08:12:46--  https://github.com/mohammed-Emad/add_faces_to_svm_class/raw/master/new_dataset.zip
Resolving github.com (github.com)... 192.30.255.112, 192.30.255.113
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mohammed-Emad/add_faces_to_svm_class/master/new_dataset.zip [following]
--2018-06-16 08:12:46--  https://raw.githubusercontent.com/mohammed-Emad/add_faces_to_svm_class/master/new_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1348055 (1.3M) [application/zip]
Saving to: ‘new_dataset.zip’

new_dataset.zip     100%[===================>]   1.29M  --.-KB/s    in 0.06s   

2018-06-16 08:12:46 (21.4 MB/s) - ‘new_dataset.z

In [0]:
#Create several new tracks only for storage
!mkdir -p align_data/train
!mkdir -p align_data/test
!mkdir -p align_data/my_new_model

#align train my dataset

In [73]:
!python align_dataset_mtcnn.py \
new_dataset/train/ \
align_data/train/ \
--image_size 160 \
--margin 32 \
--random_order \
--gpu_memory_fraction 0.25 \

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
fatal: Not a git repository (or any of the parent directories): .git
Not a git repository
To compare two paths outside a working tree:
usage: git diff [--no-index] <path> <path>
Creating networks and loading parameters
2018-06-16 08:20:29.219011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-06-16 08:20:29.219609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-16

new_dataset/train/rose_leslie/rose_leslie-8.jpg
new_dataset/train/rose_leslie/rose_leslie-16.jpg
new_dataset/train/rose_leslie/rose_leslie-11.jpg
new_dataset/train/rose_leslie/rose_leslie-1.jpg
new_dataset/train/rose_leslie/rose_leslie-7.jpg
new_dataset/train/rose_leslie/rose_leslie-4.jpg
new_dataset/train/rose_leslie/rose_leslie-9.jpg
new_dataset/train/rose_leslie/rose_leslie-5.jpg
Total number of images: 36
Number of successfully aligned images: 36


#align test my dataset

In [74]:
!python align_dataset_mtcnn.py \
new_dataset/test/ \
align_data/test/ \
--image_size 160 \
--margin 32 \
--random_order \
--gpu_memory_fraction 0.25 \

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
fatal: Not a git repository (or any of the parent directories): .git
Not a git repository
To compare two paths outside a working tree:
usage: git diff [--no-index] <path> <path>
Creating networks and loading parameters
2018-06-16 08:21:00.423321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-06-16 08:21:00.423944: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-16

new_dataset/test/Jacques_Chirac/Jacques_Chirac_0004.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0022.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0007.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0001.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0024.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0002.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0005.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0015.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0012.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0019.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0017.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0008.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0010.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0014.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0011.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0003.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0013.jpg
new_dataset/test/Jacques_Chirac/Jacques_Chirac_0

new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0018.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0006.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0012.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0011.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0002.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0009.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0017.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0020.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0004.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0022.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0015.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0007.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0014.jpg
new_dataset/test/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0005.jpg
new_dataset/test/Arn

#We will make an initial classification using the current training file
#This is before you start training on new data
#This is to clarify what is happening and what is the difference between them

In [77]:
!python my_clasifiy.py \
CLASSIFY \
align_data/test/ \
~/datasets/20180402-114759/20180402-114759.pb  \
~/my_model/lfw_classifier.pkl \
--batch_size 1000 


my_clasifiy.py:62: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-06-16 09:15:35.370111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-06-16 09:15:35.370858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-16 09:15:35.370932: I tensorflow/c

#Comment on the result of the classification without training on the latest data

**Note images from 1 to 69**

**Arnold Schwarzenegger: 0.758**

**Colin Powell: 0.766**

**Jacques Chirac: 0.721**

**Is for the three people in the old training file**

**But see for photos starting from 70**
**It's the three new people who are already wobbly in their identification, with the highest proportion being 38%{79  Arnold Schwarzenegger: 0.380}  with an unknown person**

**We follow**



#  
#  
#  Now training on new data 
**Additional training**

In [82]:
!python my_clasifiy.py \
TRAIN  \
--training_type incremental \
align_data/train/ \
~/datasets/20180402-114759/20180402-114759.pb  \
align_data/my_new_model/lfw_add_my_classifier.pkl \
--features_filename ~/my_model/all_features.pkl \
--batch_size 1000 \


my_clasifiy.py:62: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-06-16 09:35:00.920534: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-06-16 09:35:00.921239: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-16 09:35:00.921310: I tensorflow/c

**The following shows that new elements have been added to our model without retraining on all people**

**Where the number of persons in the past is 19 persons
Now the persons number is 22
The three persons were added**

In [85]:
# model all features
with open('/content/my_model/all_features.pkl', 'rb') as infile:
      (emb_array_old ,labels_old ,class_names_old) = pickle.load(infile)
    
print(len(class_names_old))


22


# This is also evident in the new model
**align_data/my_new_model/lfw_add_my_classifier.pkl**

In [90]:
# model new
with open('align_data/my_new_model/lfw_add_my_classifier.pkl', 'rb') as infile:
      (emb_array ,class_names) = pickle.load(infile)
    
print(len(class_names))
print(class_names)

22
['Ariel Sharon' 'Arnold Schwarzenegger' 'Colin Powell' 'Donald Rumsfeld'
 'George W Bush' 'Gerhard Schroeder' 'Gloria Macapagal Arroyo'
 'Hugo Chavez' 'Jacques Chirac' 'Jean Chretien' 'Jennifer Capriati'
 'John Ashcroft' 'Junichiro Koizumi' 'Laura Bush' 'Lleyton Hewitt'
 'Luiz Inacio Lula da Silva' 'Serena Williams' 'Tony Blair'
 'Vladimir Putin' 'mohamed emad' 'obama' 'rose leslie']


#Now start the test on the new test list


In [104]:
!python my_clasifiy.py \
CLASSIFY \
align_data/test/ \
~/datasets/20180402-114759/20180402-114759.pb  \
align_data/my_new_model/lfw_add_my_classifier.pkl \
--batch_size 1000 

my_clasifiy.py:62: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-06-16 11:30:19.424299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-06-16 11:30:19.424891: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-16 11:30:19.424938: I tensorflow/c

In [96]:
!ls align_data/train/bounding_boxes_29713.txt

align_data/train/bounding_boxes_29713.txt


#Here you already know the new people in addition to the people who were brought from the first training
**Maybe you now have a look at what happened
You may wonder if the level of accuracy is fairly low
For example, up to about 40% or slightly more**
**This is due to cnn and svm> you should review this
As the number of photos that I have recently uploaded is less than the pictures in the data for each person in the old DataSet**

**As the size of the features extracted from the many images of one person is greater than the images of the least person.**

**As for the accuracy which you consider to be wrong, you should consider dividing the data into training data and test data with the same number, where the number of pictures of the people here should be the same number here and this is something we have not followed now
You can see orders to divide the data into two parts**


```

--min_nrof_images_per_class 40 \
--nrof_train_images_per_class 35 \

```





**This small seed was made by**

*med emad& Sh Elabd*

**The rest of the details on [github](https://github.com/mohammed-Emad/add_faces_to_svm_class/)**
**Thanks**